# Vehicle Detection and Tracking Project

The main steps of this project are to:

1. Train a classifer to differentiate between cars and non-cars
2. Use the classifer to detect and track cars in a video

Each step will be done as follows:

#### Train a classifer

1. Select features to extract from the training set
2. Normalize the features
3. Train a classifier
4. Test the classifer

Once a classifer has been sufficiently trained and the overall accuracy is acceptable, it can be used without retraining.

#### Vehicle detection

1. Use a sliding windows of various sizes to search for positive matches
2. Convert the detected matches into a heatmap to combine multiple detections 
3. Threshold the heatmap to remove false positives
4. Draw a bounding box over the original frame of the video using the heatmap to determine location and size

---

### Imports


In [ ]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt


---

## Train a classifier

### 0. Read in the datasets

In [ ]:
def read_data():
    
    """Read in the data to train and validate the classifier
    
    Returns:
        A tuple of lists for images of vehicles and images of non-vehicles"""
    
    # Used for testing purposes while flushing out the pipeline
    VEHICLES_SMALL = 'data/vehicles_smallset/*/*.jpeg'
    NON_VEHICLES_SMALL = 'data/non-vehicles_smallset/*/*.jpeg'

    # Used for training the final model in some form or another
    VEHICLES_FULL = 'data/vehicles/*/*.png'
    NON_VEHICLES_FULL = 'data/non-vehicles/*/*.png'
    
    vehicles = glob.glob(VEHICLES_SMALL)
    non_vehicles = glob.glob(NON_VEHICLES_SMALL)

    return (vehicles, non_vehicles)


In [ ]:
cars, notcars = read_data()

### 1. Feature selection

In [ ]:
from skimage.feature import hog

In [ ]:
def convert_color(img, to_color_space='RGB', from_color_space='BGR'):

    """Helper function to convert an image from one color space to another.
    The assumption is that the image was read in using OpenCV, hence the
    'BGR' color space default.
    
    Parameters:
        • img - input image to convert
        • to_color_space - desired color space (default: 'RGB')
        • from_color_space - input color space (default: 'BGR')
        
    Returns:
        An image in the new color space or the original image if there was an error"""
    
    if to_color_space == from_color_space:
        converted_img = np.copy(img)
    else:
        try:
            # get the conversion identifier to use
            conversion = getattr(cv2, 'COLOR_{}2{}'.format(from_color_space, to_color_space))
        except AttributeError:
            return img

        # convert image to new color space (if specified)
        converted_img = cv2.cvtColor(img, conversion)

    return converted_img


In [ ]:
def bin_spatial(img, size=(32, 32)):

    """Extract the spatial binned color features
    
    Parameters:
        • img - input image
        • size - reduced size of image to use as features
        
    Returns:
        A list of color features based on the resized image"""

    # use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel()

    # Return the feature vector
    return features


In [ ]:
def color_hist(img, nbins=32, bins_range=(0, 256)):

    """Calculate a histogram for each color channel in the image and create a list of features from them.
    
    Parameters:
        • img - input image
        • nbins - number of bins in the histogram
        • bins_range - lower and upper range of the bins (above and below are ignored)
        
    Returns:
        A list of color histogram features for the image"""
    
    # Compute the histogram of the color channels separately
    if len(img.shape) > 2:
        hist_features = np.concatenate([np.histogram(img[:, :, c], bins=nbins, range=bins_range)[0] for c in range(img.shape[-1])])
    else:
        hist_features = np.array([np.histogram(img, bins=nbins, range=bins_range)[0]])
        
    # return the individual histograms, bin_centers and feature vector
    return hist_features


In [ ]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    
    """Extract the Histogram of Oriented Gradient (HOG) features for the image.
    
    Parameters:
        • img - input image
        • orient - number of orientations for HOG features
        • pix_per_cell - cell size over which each gradient histogram is computed
        • cell_per_block - specifies the local area over which the histogram counts in a given cell will be normalized
        • vis - boolean to enable a visualization of the HOG
        • feature_vec - boolean to return the data as a feature vector
        
    Returns:
        The HOG features will be returned either multidimensional or as a feature vector depending on `feature_vec`.
        If `vis` is true, an image representation of the HOG is also returned."""
    
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True,
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True,
                       visualise=vis, feature_vector=feature_vec)
        return features


In [ ]:
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9,
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    
    """Extract features from the input images based on the parameters passed in.
    
    Parameters:
        • imgs - a list of input images
        • color_space - desired color space to extract features
        • spatial_size - size for spacial binning of color features
        • hist_bins - number of bins for the color histogram features
        • orient - number of orientations for HOG features
        • pix_per_cell - cell size over which each gradient histogram is computed
        • cell_per_block - specifies the local area over which the histogram counts in a given cell will be normalized
        • hog_channel - image channel to apply the Histogram of Oriented Gradient (HOG)
        • spatial_feat - boolean to enable spatial binning of color features
        • hist_feat - boolean to enable color histogram features
        • hog_feat - boolean to enable HOG features
        
    Returns:
        A list of features per image"""
    
    # create a list to append feature vectors to
    features = []
    
    # iterate through the list of images
    for file in imgs:

        # features for this single file
        file_features = []

        # read in each one by one
        img = cv2.imread(file)

        # apply color conversion if other than 'RGB'
        feature_img = convert_color(img, color_space)
        
        # extract spatial binning of color features, if enabled
        if spatial_feat:
            spatial_features = bin_spatial(feature_img, size=spatial_size)
            file_features.append(spatial_features)
            
        # extract color histogram features, if enabled
        if hist_feat:
            hist_features = color_hist(feature_img, nbins=hist_bins)
            file_features.append(hist_features)
            
        if hog_feat:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_img[:,:,channel],
                                        orient, pix_per_cell, cell_per_block,
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)
            else:
                hog_features = get_hog_features(feature_img[:,:,hog_channel], orient,
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features


In [ ]:
car_features = extract_features(cars)
notcar_features = extract_features(cars)